In [1]:
# !pip install memory_profiler

In [2]:
%load_ext autoreload
%load_ext memory_profiler
%autoreload 2

In [3]:
import numpy as np
import pandas as pd

# Series Processing pipeline

`TODO`

# Feature extraction

The most classical way to extract features from time series is by using a **strided-window** manner.


**Challenges**:
1. Existing solutions often assume a **single** stride & window size for all features to be calculated.  
  This raises 2 problems:
   * There is no clean interface for multiple stride-window feature calculation.
   * You are responsible for the efficient execution, i.e., you need to do the bookkeeping that feature-calculations on the same stride-window pair are executed on the same time-series expansion (so that it need to be expanded only once).
2. No easy support for aggregation of multivariate series, each possibly having different sampling rates
    _(e.g., Polysomnography data, Wearable data)_
3. No efficient implementations for timestamped data (e.g., pd.Series, pd.DataFrames with a timeindex):
    * `pd.rolling` assumes same input<->output dimensions, hence no stride possible:
    See:  
    https://pandas.pydata.org/docs/reference/api/pandas.Series.rolling.html
    https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html
    * `tsfresh` has an significant memory and time overhead for `Strided-Rolling` (feature-calculation)
    https://tsfresh.readthedocs.io/en/latest/api/tsfresh.utilities.html#tsfresh.utilities.dataframe_functions.roll_time_series  
    even more:
        * There is no convenient way to retain the time-index
        * It inherently makes the single window-stride assumption for the features.  
4. Little focus on serialization (+parallelization) of local-scope objects  
   To the best of our knowledge, no existing time series library takes this into consideration, thus hampering deployment in different environments.


**What tslib does**:

= Intuitive interface for **multiple** stride-window feature calculation on multiple (possibly differently sampled) time-series signals. <a style="color:orange">_(solving 1 & 2.)_</a>

Providing the following features:
* A single registry, in which all bookkeeping is done, enabling efficient processing <a style="color:orange">(solves 2)</a>
* Maintains the time-index (after feature calculation) <a style="color:orange">(solves 3)</a>
* Serialization of `lambda's` and local-scope methods <a style="color:orange">(solves 4.)</a>

---    

**Current assumptions**:
* The time series we still use -> are sampled at fixed rates (i.e, we as assume that (and thus not check whether) there are time series gaps).
    * The Series processing serves as an option to mitigate this

**Future work(s)**:
* Big-Data: Perform feature-extraction in batch -> batch-like generators in stridedrolling objects.
* Time-wise strided rolling?

In [4]:
import sys

# Serialization
import dill as pickle
import scipy.stats as ss

# load our library
sys.path.append("time_series/")
from time_series import FeatureCollection, NumpyFuncWrapper
from time_series.features import FeatureDescriptor, MultipleFeatureDescriptors

pickle.settings["recurse"] = True  # allows to serialize lambda's YAY!

## Defining functions

Functions are defined by making use of the [NumpyFuncWrapper](time_series/features/function_wrapper.py) class.  

The `NumpyFuncWrapper` interface is easy and convienient; you define:

|      attribute 	|          type         	| info                                                     	|
|---------------:	|:---------------------:	|----------------------------------------------------------	|
|         `func` 	|        Callable       	| The wrapped function that will operate on `numpy` arrays 	|
| `output_names` 	| Union[List[str], str] 	| The name of the outputs of `func`                        	|
|     `**kwargs` 	|        Optional       	| Additional keyword-arguments for the `func`              	|

In [5]:
# --------------------- some custom feature extraction functions ---------------------
# -- 1. one-to-many functions
#    To compute quantiles, you need sort the windowed data, which is a rather expensive
#    operation n*log(n). Hence, you might want to calculate all your desired quantiles
#    in a single function-wrapper, returning multiple outputs.

quantiles = [0.25, 0.5, 0.75]
f_quantiles = NumpyFuncWrapper(
    func=np.quantile,  # the wrapped function that will operate on numpy arrays
    output_names=[f"quantile_{q}" for q in quantiles],  # the output column names
    q=quantiles,  # optional - additional function-related kwargs
)


# -- 2. in-line functions
#    You can define your functions locally; these will serialize flawleslly
def slope(x):
    return np.polyfit(np.arange(0, len(x)), x, 1)[0]


f_slope = NumpyFuncWrapper(slope, output_names="slope")

# -- 3. Lambda's
#    Or even use lambda's and other modules' functions
f_rms = NumpyFuncWrapper(lambda x: np.sqrt(np.mean(x ** 2)), output_names="rms")
f_area = NumpyFuncWrapper(np.sum, output_names="area")


# (For convenience) we store the constructed `NumpyFuncWrappers` in a list
segment_funcs = [
    np.mean,
    np.std,
    np.var,
    np.max,
    np.min,
    ss.skew,
    ss.kurtosis,
    f_quantiles,
    f_slope,
    f_rms,
    f_area,
]
segment_funcs

[<function numpy.mean(a, axis=None, dtype=None, out=None, keepdims=<no value>, *, where=<no value>)>,
 <function numpy.std(a, axis=None, dtype=None, out=None, ddof=0, keepdims=<no value>, *, where=<no value>)>,
 <function numpy.var(a, axis=None, dtype=None, out=None, ddof=0, keepdims=<no value>, *, where=<no value>)>,
 <function numpy.amax(a, axis=None, out=None, keepdims=<no value>, initial=<no value>, where=<no value>)>,
 <function numpy.amin(a, axis=None, out=None, keepdims=<no value>, initial=<no value>, where=<no value>)>,
 <function scipy.stats.stats.skew(a, axis=0, bias=True, nan_policy='propagate')>,
 <function scipy.stats.stats.kurtosis(a, axis=0, fisher=True, bias=True, nan_policy='propagate')>,
 NumpyFuncWrapper(quantile, ['quantile_0.25', 'quantile_0.5', 'quantile_0.75'], {'q': [0.25, 0.5, 0.75]}),
 NumpyFuncWrapper(slope, ['slope'], {}),
 NumpyFuncWrapper(<lambda>, ['rms'], {}),
 NumpyFuncWrapper(sum, ['area'], {})]

## Use case: single series feature extraction

The defined functions above will be encapsulated in a [FeatureDescriptor](time_series/features/feature.py) object.

A `FeatureDescriptor` describes a feature, and has 4 main attributes:

|  attribute 	|                  type                 	| info                                                                                                             	|
|-----------:	|:-------------------------------------:	|------------------------------------------------------------------------------------------------------------------	|
| `function` 	| Union[Callable, <br>NumpyFuncWrapper] 	| The `function` that calculates this feature.                                                                     	|
|      `key` 	|                 string                	| The signal key; i.e., the `pd.DataFrame` column name or <br> `pd.Series` name on which the function will operate.     	|
|   `window` 	|                  int                  	| The window size on which this feature will be applied, <br> expressed in the number of samples from the input signal. 	|
|   `stride` 	|                  int                  	| The stride of the window rolling process, also as a <br> number of samples of the input signal.                       	|

**note**: [MultipleFeatureDescriptor](time_series/features/feature.py) is actaully a factory for `FeatureDescriptor` objects.

### Example 1: Fixed window size and strides

**note**: this functionality is exposed by most existing time-series libraries.

In this example, we will use the _temperature_ signal from a wearable

In [6]:
# define the sample frequency and window size
fs_tmp = 4  # 4Hz
tmp_win_size = 60 * fs_tmp  # window of 60s
tmp_stride_size = 30 * fs_tmp  # stride of 30s


tmp_feat_extr = FeatureCollection(
    feature_desc_list=[
        MultipleFeatureDescriptors(
            signal_keys=["TMP"],
            windows=[tmp_win_size],
            strides=[tmp_stride_size],
            functions=segment_funcs,  # The list of functions we constructed earlier
        )
    ]
)
tmp_feat_extr

TMP: (
	win: 240 samples, stride: 120 samples: [
		FeatureDescriptor - func: NumpyFuncWrapper(mean, ['mean'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(std, ['std'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(var, ['var'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(amax, ['amax'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(amin, ['amin'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(skew, ['skew'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(kurtosis, ['kurtosis'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(quantile, ['quantile_0.25', 'quantile_0.5', 'quantile_0.75'], {'q': [0.25, 0.5, 0.75]}),
		FeatureDescriptor - func: NumpyFuncWrapper(slope, ['slope'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(<lambda>, ['rms'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(sum, ['area'], {}),
	]
)

In [7]:
df_tmp = pd.read_feather("data/tmp.feather").set_index("timestamp")
df_tmp.sample(2)

,TMP
timestamp,
2017-06-13 12:17:58+02:00,32.07
2017-06-13 11:17:51.750000+02:00,33.05


* **TODO**: describe output column names

In [8]:
%%time
# not multiprocessing
tmp_feat_extr.calculate(df_tmp, merge_dfs=True, n_jobs=1).sample(2)

CPU times: user 377 ms, sys: 7.22 ms, total: 385 ms
Wall time: 380 ms


,TMP_mean__w=240_s=120,TMP_std__w=240_s=120,TMP_var__w=240_s=120,TMP_amax__w=240_s=120,TMP_amin__w=240_s=120,TMP_skew__w=240_s=120,TMP_kurtosis__w=240_s=120,TMP_quantile_0.25__w=240_s=120,TMP_quantile_0.5__w=240_s=120,TMP_quantile_0.75__w=240_s=120,TMP_slope__w=240_s=120,TMP_rms__w=240_s=120,TMP_area__w=240_s=120
timestamp,,,,,,,,,,,,,
2017-06-13 12:30:39.750000+02:00,31.589333,0.021899,0.000480,31.63,31.55,-0.010214,-0.641316,31.57,31.59,31.61,-0.000256,31.589341,7581.44
2017-06-13 11:41:09.750000+02:00,32.972667,0.014360,0.000206,32.99,32.95,-0.203304,-1.050535,32.97,32.97,32.99,0.000119,32.972670,7913.44


In [9]:
%%time
# not multiprocessing
tmp_feat_extr.calculate(df_tmp, merge_dfs=True, n_jobs=0).sample(2)

CPU times: user 375 ms, sys: 7.59 ms, total: 383 ms
Wall time: 378 ms


,TMP_mean__w=240_s=120,TMP_std__w=240_s=120,TMP_var__w=240_s=120,TMP_amax__w=240_s=120,TMP_amin__w=240_s=120,TMP_skew__w=240_s=120,TMP_kurtosis__w=240_s=120,TMP_quantile_0.25__w=240_s=120,TMP_quantile_0.5__w=240_s=120,TMP_quantile_0.75__w=240_s=120,TMP_slope__w=240_s=120,TMP_rms__w=240_s=120,TMP_area__w=240_s=120
timestamp,,,,,,,,,,,,,
2017-06-13 11:30:09.750000+02:00,32.804500,0.020930,0.000438,32.87,32.77,0.593804,-0.112342,32.79,32.79,32.83,-0.000234,32.804507,7873.08
2017-06-13 12:27:39.750000+02:00,31.662333,0.015096,0.000228,31.69,31.63,0.065521,-0.424477,31.65,31.67,31.67,0.000103,31.662337,7598.96


In [10]:
%%time
# multiprocessing
tmp_feat_extr.calculate(df_tmp, merge_dfs=True).sample(2)

  0%|          | 0/11 [00:00<?, ?it/s]

CPU times: user 142 ms, sys: 129 ms, total: 271 ms
Wall time: 455 ms


,TMP_mean__w=240_s=120,TMP_amax__w=240_s=120,TMP_std__w=240_s=120,TMP_amin__w=240_s=120,TMP_var__w=240_s=120,TMP_area__w=240_s=120,TMP_rms__w=240_s=120,TMP_quantile_0.25__w=240_s=120,TMP_quantile_0.5__w=240_s=120,TMP_quantile_0.75__w=240_s=120,TMP_slope__w=240_s=120,TMP_kurtosis__w=240_s=120,TMP_skew__w=240_s=120
timestamp,,,,,,,,,,,,,
2017-06-13 12:19:09.750000+02:00,31.963833,32.00,0.022143,31.91,0.000490,7671.32,31.963841,31.95,31.97,31.99,-0.000246,-0.778842,-0.065536
2017-06-13 10:42:39.750000+02:00,31.867000,31.93,0.026539,31.81,0.000704,7648.08,31.867011,31.85,31.87,31.89,0.000304,-0.286737,-0.237207


In [11]:
%%memit
tmp_feat_extr.calculate(df_tmp, merge_dfs=True, show_progress=False)

peak memory: 143.79 MiB, increment: 1.40 MiB


In [12]:
%%memit
tmp_feat_extr.calculate(df_tmp, merge_dfs=False, show_progress=False)

peak memory: 145.57 MiB, increment: 1.27 MiB


### Example 2: Using multiple `time-based` window sizes and strides

In this example, we will use multiple stride-window-size combinations on a wearables' **Galvanic Skin Response** (GSR) signal , also known ElectorDermal Activity (EDA). 

Note that we do not user int-based window-stride combinations, but `time-based` ones. Also take a look at the `__repr__` string

In [13]:
# PoC: we will select a random combination of the window_size stride combination
window_size_s = ['30s', '120s', '90s', '1h']
stride_size_s = ['15s', '30s']

import random

gsr_feat_extr = FeatureCollection(
    [
        FeatureDescriptor(
            key="EDA",
            window=random.choice(window_size_s),
            stride=random.choice(stride_size_s),
            function=f,
        )
        for f in segment_funcs
    ]
)
gsr_feat_extr

EDA: (
	win: 1h    , stride: 30s: [
		FeatureDescriptor - func: NumpyFuncWrapper(mean, ['mean'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(amin, ['amin'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(kurtosis, ['kurtosis'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(quantile, ['quantile_0.25', 'quantile_0.5', 'quantile_0.75'], {'q': [0.25, 0.5, 0.75]}),
		FeatureDescriptor - func: NumpyFuncWrapper(sum, ['area'], {}),
	]
	win: 2m    , stride: 30s: [
		FeatureDescriptor - func: NumpyFuncWrapper(std, ['std'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(var, ['var'], {}),
	]
	win: 30s   , stride: 15s: [
		FeatureDescriptor - func: NumpyFuncWrapper(amax, ['amax'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(<lambda>, ['rms'], {}),
	]
	win: 1m30s , stride: 30s: [
		FeatureDescriptor - func: NumpyFuncWrapper(skew, ['skew'], {}),
	]
	win: 1m30s , stride: 15s: [
		FeatureDescriptor - func: NumpyFuncWrapper(slope, ['slope'], {}),
	]
)

In [14]:
df_gsr = pd.read_feather("data/gsr.feather").set_index("timestamp")
df_gsr.sample(2)

,EDA
timestamp,
2017-06-13 11:13:35.750000+02:00,0.117951
2017-06-13 11:31:20.250000+02:00,0.199818


In [15]:
%%time
gsr_feat_extr.calculate(df_gsr, merge_dfs=True, show_progress=False, n_jobs=0).sample(2)

CPU times: user 447 ms, sys: 2.99 ms, total: 450 ms
Wall time: 450 ms


,EDA_mean__w=1h_s=30s,EDA_amin__w=1h_s=30s,EDA_kurtosis__w=1h_s=30s,EDA_quantile_0.25__w=1h_s=30s,EDA_quantile_0.5__w=1h_s=30s,EDA_quantile_0.75__w=1h_s=30s,EDA_area__w=1h_s=30s,EDA_std__w=2m_s=30s,EDA_var__w=2m_s=30s,EDA_amax__w=30s_s=15s,EDA_rms__w=30s_s=15s,EDA_skew__w=1m30s_s=30s,EDA_slope__w=1m30s_s=15s
timestamp,,,,,,,,,,,,,
2017-06-13 12:08:24.750000+02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.333848,2.131858,NaN,0.000055
2017-06-13 11:25:54.750000+02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.211330,0.202964,NaN,-0.000093


In [16]:
%%memit
gsr_feat_extr.calculate(df_gsr, merge_dfs=True)

  0%|          | 0/11 [00:00<?, ?it/s]

peak memory: 180.20 MiB, increment: 32.48 MiB


## Use case: Multiple series feature extraction

In [17]:
# Construct the feature FeatureCollection
#   =  higher order wrapper which aggregates the featuredescriptions
multimodal_feature_extraction = FeatureCollection(
    feature_desc_list=[gsr_feat_extr, tmp_feat_extr]
)
multimodal_feature_extraction

EDA: (
	win: 1h    , stride: 30s: [
		FeatureDescriptor - func: NumpyFuncWrapper(mean, ['mean'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(amin, ['amin'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(kurtosis, ['kurtosis'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(quantile, ['quantile_0.25', 'quantile_0.5', 'quantile_0.75'], {'q': [0.25, 0.5, 0.75]}),
		FeatureDescriptor - func: NumpyFuncWrapper(sum, ['area'], {}),
	]
	win: 2m    , stride: 30s: [
		FeatureDescriptor - func: NumpyFuncWrapper(std, ['std'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(var, ['var'], {}),
	]
	win: 30s   , stride: 15s: [
		FeatureDescriptor - func: NumpyFuncWrapper(amax, ['amax'], {}),
		FeatureDescriptor - func: NumpyFuncWrapper(<lambda>, ['rms'], {}),
	]
	win: 1m30s , stride: 30s: [
		FeatureDescriptor - func: NumpyFuncWrapper(skew, ['skew'], {}),
	]
	win: 1m30s , stride: 15s: [
		FeatureDescriptor - func: NumpyFuncWrapper(slope, ['slope'], {}),
	]
)
TMP: (
	win: 240 samples, str

In [18]:
%%memit
multimodal_feature_extraction.calculate([df_gsr, df_tmp], merge_dfs=True)

  0%|          | 0/22 [00:00<?, ?it/s]

peak memory: 184.14 MiB, increment: 32.34 MiB


In [19]:
%%time
df_feat = multimodal_feature_extraction.calculate([df_gsr, df_tmp], merge_dfs=True)
df_feat.sample(2)

  0%|          | 0/22 [00:00<?, ?it/s]

CPU times: user 369 ms, sys: 268 ms, total: 638 ms
Wall time: 877 ms


,EDA_mean__w=1h_s=30s,EDA_amin__w=1h_s=30s,EDA_kurtosis__w=1h_s=30s,EDA_quantile_0.25__w=1h_s=30s,EDA_quantile_0.5__w=1h_s=30s,EDA_quantile_0.75__w=1h_s=30s,EDA_std__w=2m_s=30s,EDA_amax__w=30s_s=15s,EDA_var__w=2m_s=30s,EDA_area__w=1h_s=30s,...,TMP_area__w=240_s=120,TMP_rms__w=240_s=120,TMP_quantile_0.25__w=240_s=120,TMP_quantile_0.5__w=240_s=120,TMP_quantile_0.75__w=240_s=120,TMP_slope__w=240_s=120,EDA_slope__w=1m30s_s=15s,EDA_skew__w=1m30s_s=30s,TMP_kurtosis__w=240_s=120,TMP_skew__w=240_s=120
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-06-13 10:36:39.750000+02:00,NaN,NaN,NaN,NaN,NaN,NaN,0.206870,0.634737,0.042795,NaN,...,7543.44,31.431043,31.385,31.43,31.47,0.000728,0.001046,-1.037737,-1.316087,0.035291
2017-06-13 11:45:09.750000+02:00,0.166868,0.065504,30.03864,0.120509,0.147372,0.174234,0.001831,0.156326,0.000003,2402.8931,...,7900.72,32.919669,32.910,32.92,32.93,-0.000103,0.000010,-0.433320,-0.253140,-0.124094


## Serialization

Serialization is mandatory to store and share your pipelines.
`TODO`

In [20]:
multimodal_feature_extraction.serialize("data/example_serialization.pkl")

## Other packages

### tsfresh

https://tsfresh.readthedocs.io/en/latest/api/tsfresh.utilities.html#tsfresh.utilities.dataframe_functions.roll_time_series

In [ ]:
from tsfresh.feature_extraction import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series

In [ ]:
# define the window-size and stride
# used the largest window and smallest strided, defined above
window = 480
stride = 40

df_gsr_id = df_gsr.reset_index(drop=False).copy()  # .set_index('timestamp', drop=True)
df_gsr_id["id"] = 1
df_gsr_id.sample(2)

**Note**: This ouputs merely one expansion with a fixed window and stride.

In [21]:
%%memit
tsf_out = roll_time_series(
    df_gsr_id,
    column_id="id",
    max_timeshift=window,
    min_timeshift=window,
    rolling_direction=stride,
)

Rolling: 100%|██████████| 80/80 [00:01<00:00, 51.69it/s]


peak memory: 373.57 MiB, increment: 133.35 MiB


In [22]:
%%time
roll_time_series(
    df_gsr_id.reset_index(drop=True),
    column_id="id",
    max_timeshift=window,
    min_timeshift=window,
    rolling_direction=stride,
).sample(2)

Rolling: 100%|██████████| 80/80 [00:01<00:00, 50.77it/s]


CPU times: user 1.89 s, sys: 442 ms, total: 2.33 s
Wall time: 2.34 s


,timestamp,EDA,id,sort
81134,2017-06-13 11:03:10.750000+02:00,0.211330,"(1, 7237)",7083
199035,2017-06-13 11:44:14.750000+02:00,0.151209,"(1, 17037)",16939


In [23]:
%%memit
tsf_feats = extract_features(tsf_out.drop(columns="timestamp"), column_id="id")

Feature Extraction: 100%|██████████| 79/79 [01:13<00:00,  1.08it/s]


peak memory: 702.43 MiB, increment: 305.52 MiB


In [24]:
# some logic re-needed to add timestamp to features

In [25]:
%%time
extract_features(
    roll_time_series(
        df_gsr_id.reset_index(drop=True),
        column_id="id",
        max_timeshift=window,
        min_timeshift=window,
        rolling_direction=stride,
    ).drop(columns="timestamp"),
    column_id="id",
).sample(2)

Feature Extraction: 100%|██████████| 79/79 [01:13<00:00,  1.08it/s]


CPU times: user 5.25 s, sys: 1.45 s, total: 6.7 s
Wall time: 1min 17s


EDA__variance_larger_than_standard_deviation  EDA__has_duplicate_max  \
1 30957                                           0.0                     0.0   
  19877                                           0.0                     0.0   

         EDA__has_duplicate_min  EDA__has_duplicate  EDA__sum_values  \
1 30957                     0.0                 1.0        71.245141   
  19877                     1.0                 1.0       111.884515   

         EDA__abs_energy  EDA__mean_abs_change  EDA__mean_change  \
1 30957        10.605110              0.001655         -0.000061   
  19877        26.125586              0.002033          0.000008   

         EDA__mean_second_derivative_central  EDA__median  ...  \
1 30957                             0.000004     0.146092  ...   
  19877                             0.000005     0.235635  ...   

         sort__permutation_entropy__dimension_5__tau_1  \
1 30957                                           -0.0   
  19877                                           -0.0   

         sort__permutation_entropy__dimension_6__tau_1  \
1 30957                                           -0.0   
  19877                                           -0.0   

         sort__permutation_entropy__dimension_7__tau_1  \
1 30957                                           -0.0   
  19877                                           -0.0   

         sort__query_similarity_count__query_None__threshold_0.0  \
1 30957                                                NaN         
  19877                                                NaN         

         sort__matrix_profile__feature_"min"__threshold_0.98  \
1 30957                                                0.0     
  19877                                                0.0     

         sort__matrix_profile__feature_"max"__threshold_0.98  \
1 30957                                                0.0     
  19877                                                0.0     

         sort__matrix_profile__feature_"mean"__threshold_0.98  \
1 30957                                                0.0      
  19877                                                0.0      

         sort__matrix_profile__feature_"median"__threshold_0.98  \
1 30957                                                0.0        
  19877                                                0.0        

         sort__matrix_profile__feature_"25"__threshold_0.98  \
1 30957                                                0.0    
  19877                                                0.0    

         sort__matrix_profile__feature_"75"__threshold_0.98  
1 30957                                                0.0   
  19877                                                0.0   

[2 rows x 1574 columns]

### Seglearn

https://tsfresh.readthedocs.io/en/latest/api/tsfresh.utilities.html#tsfresh.utilities.dataframe_functions.roll_time_series

In [ ]:
# !pip install -U seglearn

In [27]:
from numpy.random import rand
from seglearn.pipe import Pype
from seglearn.transform import FeatureRep, Segment
from seglearn.base import TS_Data
from seglearn.util import ts_stats, check_ts_data

In [28]:
s = Segment(width=480, step=40, order="F")

In [29]:
%%time
s.fit_transform(np.column_stack(df_gsr['EDA']), y=None)
s.transform(np.column_stack(df_tmp['TMP']), y=None)

CPU times: user 216 ms, sys: 9.13 ms, total: 225 ms
Wall time: 223 ms


(array([[382.21, 382.21, 382.21, ...,  31.35,  31.35,  31.35],
        [ 31.13,  31.13,  31.13, ...,  31.37,  31.37,  31.37],
        [ 31.15,  31.15,  31.15, ...,  31.37,  31.37,  31.37],
        ...,
        [ 31.39,  31.39,  31.39, ...,  31.35,  31.35,  31.35],
        [ 31.41,  31.41,  31.41, ...,  31.37,  31.37,  31.37],
        [ 31.39,  31.39,  31.39, ...,  31.35,  31.35,  31.35]]),
 None,
 None)

Speed seems to be in the same magnitude, but the time index is gone.

In [30]:
f_extr_pype = Pype([
    ("segment", Segment(width=480, step=40, order="F")),
    ("features", FeatureRep(features="default")),
])

/users/jonvdrdo/jonas/projects/context_aware_health_monitoring/.caw_venv37/lib/python3.7/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass memory=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [33]:
f_extr_pype.fit_transform(np.column_stack(df_gsr['EDA']), y=None)

TypeError: object of type 'NoneType' has no len()

`TODO`: https://dmbee.github.io/seglearn/auto_examples/plot_feature_rep_mix_example.html#sphx-glr-auto-examples-plot-feature-rep-mix-example-py
still need to further look into this

# Serialization

## Series Pipeline

`TODO`

## Feature extraction

In [ ]:
# restart the kernel
import os

os._exit(0)

In [1]:
import pickle
import sys

import pandas as pd

time_series_dir = "../time_series/"
data_dir = "data/"

sys.path.append(time_series_dir)

In [2]:
with open(f"data/example_serialization.pkl", "rb") as f:
    multimodal_feature_extraction = pickle.load(f)

df_gsr = pd.read_feather(f"{data_dir}gsr.feather").set_index("timestamp")
df_tmp = pd.read_feather(f"{data_dir}tmp.feather").set_index("timestamp")

**note**: This is truly amazing, we do not need redefine which local funcs were used;  
We only need a python kernel which knows the paths to the modules that are used in the serialization.

In [3]:
df_feat = multimodal_feature_extraction.calculate([df_gsr, df_tmp], merge_dfs=True)
df_feat.sample(2)

,EDA_mean__w=1m30s_s=15s,EDA_slope__w=1m30s_s=15s,EDA_var__w=2m_s=30s,EDA_rms__w=2m_s=30s,EDA_area__w=2m_s=30s,EDA_std__w=1h_s=30s,EDA_amax__w=1h_s=15s,EDA_amin__w=30s_s=30s,TMP_mean__w=240_s=120,TMP_var__w=240_s=120,...,EDA_kurtosis__w=30s_s=30s,TMP_slope__w=240_s=120,TMP_quantile_0.25__w=240_s=120,TMP_quantile_0.5__w=240_s=120,TMP_quantile_0.75__w=240_s=120,EDA_quantile_0.25__w=1h_s=15s,EDA_quantile_0.5__w=1h_s=15s,EDA_quantile_0.75__w=1h_s=15s,TMP_kurtosis__w=240_s=120,TMP_skew__w=240_s=120
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-06-13 12:03:39.750000+02:00,2.423588,-0.000846,0.027532,2.477573,1186.564884,0.624615,3.111371,2.213085,32.472333,0.000315,...,-0.592603,-0.000173,32.47,32.47,32.49,0.125626,0.158884,0.183188,0.114720,-0.624008
2017-06-13 12:20:39.750000+02:00,0.344951,-0.000151,0.000675,0.358119,171.443896,0.854842,3.119045,0.317502,31.963333,0.000262,...,-0.828058,0.000094,31.95,31.97,31.97,0.157605,0.190863,0.874332,-0.526573,-0.076756


# Bonus - Get LAYD: Look At Your Data

And as a bonus, for running/reading this notebook, you get some nice visualization code, for
ofcourse time-series.

In [4]:
import ipywidgets as widgets
import plotly.graph_objects as go
from ipywidgets import interact_manual
from plotly.subplots import make_subplots

In [5]:
df_dict = {"tmp": df_tmp, "gsr": df_gsr}

In [6]:
feat_widget = widgets.SelectMultiple(options=df_feat.columns)
sig_widget = widgets.SelectMultiple(options=["gsr", "tmp"])

In [7]:
@interact_manual
def visuzalize(features=feat_widget, signals=sig_widget):
    row_titles = list(signals) + ["features"] if len(features) else []
    fig = make_subplots(
        rows=len(row_titles),
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1 / len(row_titles),
        row_titles=row_titles,
    )
    fig.update_layout(height=300 * len(row_titles))

    # first, visualize the "raw" signals
    row_idx = 1
    for sig in signals:
        df_sig = df_dict[sig][10:].resample("1s").mean()
        for col in set(df_sig.columns).difference(["index", "timestamp"]):
            fig.add_trace(
                go.Scattergl(x=df_sig.index, y=df_sig[col], name=col, hoverinfo="skip"),
                row=row_idx,
                col=1,
            )
        row_idx += 1

    # then visualize the features
    for feature in features:
        df_ff = df_feat[feature].dropna()
        fig.add_trace(
            go.Scattergl(
                connectgaps=True,
                x=df_ff.index,
                y=df_ff,
                name=feature,
                hoverinfo="skip",
                mode="markers",
                showlegend=True,
            ),
            row=row_idx,
            col=1,
        )

    return fig

interactive(children=(SelectMultiple(description='features', options=('EDA_mean__w=1m30s_s=15s', 'EDA_slope__w…